In [5]:
import requests

In [35]:
url = 'https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/UK/GBP/en-GB/SFO-sky/NYCA-sky/2020-12-12'
params = {'inboundpartialdate':'2020-12-12'}
headers = {
    'x-rapidapi-host': 'skyscanner-skyscanner-flight-search-v1.p.rapidapi.com',
    'x-rapidapi-key': '6afdbd75b3mshc576d2af9d6461dp15eb73jsna7873b9d3591'}
response = requests.get(url, headers=headers, params=params)
response.json()


response.json()

{'Quotes': [{'QuoteId': 1,
   'MinPrice': 83,
   'Direct': True,
   'OutboundLeg': {'CarrierIds': [851],
    'OriginId': 81727,
    'DestinationId': 50290,
    'DepartureDate': '2020-12-12T00:00:00'},
   'QuoteDateTime': '2020-12-01T09:33:00'}],
 'Carriers': [{'CarrierId': 851, 'Name': 'Alaska Airlines'}],
 'Places': [{'Name': 'New York Newark',
   'Type': 'Station',
   'PlaceId': 50290,
   'IataCode': 'EWR',
   'SkyscannerCode': 'EWR',
   'CityName': 'New York',
   'CityId': 'NYCA',
   'CountryName': 'United States'},
  {'Name': 'San Francisco International',
   'Type': 'Station',
   'PlaceId': 81727,
   'IataCode': 'SFO',
   'SkyscannerCode': 'SFO',
   'CityName': 'San Francisco',
   'CityId': 'SFOA',
   'CountryName': 'United States'}],
 'Currencies': [{'Code': 'GBP',
   'Symbol': '£',
   'ThousandsSeparator': ',',
   'DecimalSeparator': '.',
   'SymbolOnLeft': True,
   'SpaceBetweenAmountAndSymbol': False,
   'RoundingCoefficient': 0,
   'DecimalDigits': 2}]}

In [36]:
import pandas as pd
import json

In [37]:
# 1. Step normailize all oclolumns in to data frame
pd.json_normalize(json_sky)

,Quotes,Carriers,Places,Currencies
0,"[{'QuoteId': 1, 'MinPrice': 83, 'Direct': True...","[{'CarrierId': 851, 'Name': 'Alaska Airlines'}]","[{'Name': 'New York Newark', 'Type': 'Station'...","[{'Code': 'GBP', 'Symbol': '£', 'ThousandsSepa..."


In [39]:
# 2. normalize each column 

quotes = pd.DataFrame(pd.json_normalize(response.json())['Quotes'][0])
quotes

,QuoteId,MinPrice,Direct,OutboundLeg,QuoteDateTime
0,1,83,True,"{'CarrierIds': [851], 'OriginId': 81727, 'Dest...",2020-12-01T09:33:00


In [41]:

carriers = pd.DataFrame(pd.json_normalize(response.json())['Carriers'][0])
carriers

,CarrierId,Name
0,851,Alaska Airlines


In [42]:
places = pd.DataFrame(pd.json_normalize(response.json())['Places'][0])
places

,Name,Type,PlaceId,IataCode,SkyscannerCode,CityName,CityId,CountryName
0,New York Newark,Station,50290,EWR,EWR,New York,NYCA,United States
1,San Francisco International,Station,81727,SFO,SFO,San Francisco,SFOA,United States


In [43]:
flights = pd.DataFrame(pd.DataFrame(pd.json_normalize(response.json())['Quotes'][0])['OutboundLeg'][0])
flights

,CarrierIds,OriginId,DestinationId,DepartureDate
0,851,81727,50290,2020-12-12T00:00:00


In [44]:
# 3. Creating function that retrieves lowest price per date

def lowest_price(origin, destination, start, end):
    flights = flight_days(origin, destination, start, end)
    prices = {date:min([quote["MinPrice"] for quote in flights[date]["Quotes"]]) for date in flights for quote in flights[date]["Quotes"]}
    return min(prices, key = prices.get)